NeuralChat is a customizable chat framework designed to create user own chatbot within few minutes on multiple architectures. This notebook is used to demonstrate how to build a talking chatbot on 4th Generation of Intel® Xeon® Scalable Processors Sapphire Rapids.

The 4th Generation of Intel® Xeon® Scalable processor provides two instruction sets viz. AMX_BF16 and AMX_INT8 which provides acceleration for bfloat16 and int8 operations respectively.

# Prepare Environment

Install intel extension for transformers:

In [1]:
!pip install intel-extension-for-transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Install Requirements:

In [2]:
!git clone https://github.com/intel/intel-extension-for-transformers.git

fatal: destination path 'intel-extension-for-transformers' already exists and is not an empty directory.


In [3]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/
!pip install -r requirements_cpu.txt
%cd ../../../

/home/u3ec174e93a7f0c7fe660f9bf3c99e8a/intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/docs/notebooks/intel-extension-for-transformers/intel_extension_for_transformers/neural_chat


/home/u3ec174e93a7f0c7fe660f9bf3c99e8a/myenv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pytorch-extension.intel.com/release-whl/stable/cpu/us/
  Using cached fschat-0.2.32-py3-none-any.whl.metadata (20 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of transformers[sentencepiece] to determine which version is compatible with other requirements. This could take a while.
Using cached fschat-0.2.32-py3-none-any.whl (211 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transf

# Build your chatbot 💻

## Text Chat

Giving NeuralChat the textual instruction, it will respond with the textual response.

In [6]:
# BF16 Optimization
from intel_extension_for_transformers.neural_chat import build_chatbot, PipelineConfig
from intel_extension_for_transformers.transformers import MixedPrecisionConfig
config = PipelineConfig(optimization_config=MixedPrecisionConfig())
chatbot = build_chatbot(config)
response = chatbot.predict(query="How can transformer-based models be leveraged to improve the accuracy and interpretability of sentiment analysis in multilingual contexts?")
print(response)


/home/u3ec174e93a7f0c7fe660f9bf3c99e8a/myenv/lib/python3.9/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/u3ec174e93a7f0c7fe660f9bf3c99e8a/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading model Intel/neural-chat-7b-v3-1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In multilingual contexts, transformer-based models can be utilized to enhance the accuracy and interpretability of sentiment analysis through various techniques. Here's how:

1. Multilingual pre-training: Train transformer models on large datasets containing multiple languages, which helps them learn common patterns across different languages. This improves their ability to generalize and adapt to new languages.

2. Language transfer learning: Transfer knowledge from one language to another by fine-tuning the model with data from the target language. This allows the model to better understand the nuances of each language while still benefiting from the broader understanding gained during pre-training.

3. Cross-lingual representation learning: Develop shared representations for words or phrases across languages, enabling the model to recognize similarities between related terms in different languages. This can lead to improved performance when dealing with diverse linguistic expression

## Text Chat With Retrieval Plugin

User could also leverage NeuralChat Retrieval plugin to do domain specific chat by feding with some documents like below:

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/pipeline/plugins/retrieval/
!pip install -r requirements.txt
%cd ../../../../../../

In [ ]:
!mkdir docs
%cd docs
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.jsonl
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.txt
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.xlsx
%cd ..

In [5]:
from intel_extension_for_transformers.neural_chat import PipelineConfig
from intel_extension_for_transformers.neural_chat import build_chatbot
from intel_extension_for_transformers.neural_chat import plugins
plugins.retrieval.enable=True
plugins.retrieval.args["input_path"]="./docs/"
config = PipelineConfig(plugins=plugins)
chatbot = build_chatbot(config)
response = chatbot.predict("How many cores does the Intel® Xeon® Platinum 8480+ Processor have in total?")
print(response)

2024-07-07 12:53:00,926 - error_utils.py - intel_extension_for_transformers.neural_chat.utils.error_utils - ERROR - neuralchat error: Generic error
2024-07-07 12:53:00,928 - chatbot.py - intel_extension_for_transformers.neural_chat.chatbot - ERROR - build_chatbot: plugin init failed


create retrieval plugin instance...
plugin parameters:  {'input_path': './docs/'}


AttributeError: 'NoneType' object has no attribute 'predict'

## Voice Chat with ASR & TTS Plugin

In the context of voice chat, users have the option to engage in various modes: utilizing input audio and receiving output audio, employing input audio and receiving textual output, or providing input in textual form and receiving audio output.

For the Python API code, users have the option to enable different voice chat modes by setting ASR and TTS plugins enable or disable.

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/pipeline/plugins/audio/
!pip install -r requirements.txt
%cd ../../../../../../

In [ ]:
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/speaker_embeddings/spk_embed_default.pt
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/audio/sample.wav

In [ ]:
from intel_extension_for_transformers.neural_chat import PipelineConfig
from intel_extension_for_transformers.neural_chat import build_chatbot
from intel_extension_for_transformers.neural_chat import plugins
plugins.tts.enable = True
plugins.tts.args["output_audio_path"] = "./response.wav"
plugins.asr.enable = True

config = PipelineConfig(plugins=plugins)
chatbot = build_chatbot(config)
result = chatbot.predict(query="./sample.wav")
print(result)

# Low Precision Optimization

## BF16

In [4]:
# BF16 Optimization
from intel_extension_for_transformers.neural_chat.config import PipelineConfig
from intel_extension_for_transformers.transformers import MixedPrecisionConfig
config = PipelineConfig(optimization_config=MixedPrecisionConfig())
chatbot = build_chatbot(config)
response = chatbot.predict(query="Tell me about Intel Xeon Scalable Processors.")
print(response)

ModuleNotFoundError: No module named 'intel_extension_for_transformers'